In [1]:
import rasterio
from rasterio.features import shapes
import numpy as np
import geopandas as gpd
from shapely.geometry import shape

tif_path = "../data/raw/climate/chirps/chirps-v2.0.2015.1201.tiff"

with rasterio.open(tif_path) as src:
    img = src.read(1)                    # 1번 밴드
    nodata = src.nodata
    transform = src.transform
    crs = src.crs

    # nodata 마스킹
    mask = None
    if nodata is not None:
        mask = img != nodata

    # (중요) 연속값이면 먼저 임계값/분류로 범주화하세요. 아래는 예시:
    # img_bin = (img > 0.5).astype(np.uint8)  # 연속값 → 0/1
    # data = img_bin
    data = img

    geoms = []
    vals = []

    for geom, val in shapes(data, mask=mask, transform=transform):
        # 필요 시 배경값/NaN 걸러내기
        if nodata is not None and (val == nodata or (isinstance(val, float) and np.isnan(val))):
            continue
        geoms.append(shape(geom))
        vals.append(val)

gdf = gpd.GeoDataFrame({"value": vals, "geometry": geoms}, crs=crs)
gdf = gdf.explode(index_parts=False).reset_index(drop=True)  # 멀티폴리곤 분해(선택)
# gdf.to_file("vectorized.gpkg", driver="GPKG")

KeyboardInterrupt: 